In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, broadcast, current_timestamp, regexp_replace
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, TimestampType

# Inicialización de SPARK

In [9]:
# Paquetes requeridos para Kafka y Delta Lake (deben ser coherentes con la versión de Spark 3.5.1)
SPARK_PACKAGES = (
    "io.delta:delta-spark_2.12:3.1.0,"
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1"
)
SPARK_MASTER = "spark://0.0.0.0:7077" 

# Configuración de Spark Session
spark = SparkSession.builder \
    .appName("BronzeToSilverStreaming") \
    .master(SPARK_MASTER) \
    .config("spark.jars.packages", SPARK_PACKAGES) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
print("Sesión de Spark iniciada con soporte para Kafka y Delta Lake.")

Sesión de Spark iniciada con soporte para Kafka y Delta Lake.


# Ingesta: Leer stream de Kafka

In [10]:
# Esquema de los datos reales del productor (8 campos clave + 6 auxiliares)
CONTRACT_SCHEMA = StructType([
    # Campos clave para ML
    StructField("id_contrato", StringType(), True),
    StructField("objeto_contrato", StringType(), True),
    StructField("entidad", StringType(), True),
    StructField("codigo_unspsc", StringType(), True), 
    StructField("duracion_dias", LongType(), True),
    StructField("valor_contrato", DoubleType(), True),
    StructField("fecha_firma", StringType(), True),
    StructField("departamento", StringType(), True),
    
    # Columnas ruidosas / auxiliares (Serán eliminadas)
    StructField("nit_entidad", StringType(), True), 
    StructField("localizacion", StringType(), True),
    StructField("sector", StringType(), True), 
    StructField("es_pyme", StringType(), True),
    StructField("valor_facturado", StringType(), True), # Aunque es numérico, puede venir sucio
    StructField("urlproceso", StringType(), True),
])

# 1. Tarea: Leer el stream de Kafka
kafka_stream = (spark.readStream
    .format("kafka") 
    .option("kafka.bootstrap.servers", "kafka:29092")
    .option("subscribe", "contratos-publicos")
    .option("startingOffsets", "earliest") # Para procesar datos desde el inicio (si el productor ya corrió)
    .load()
)

print("Stream de Kafka configurado.")

Stream de Kafka configurado.


# Preparación de los datos

In [11]:
# Definimos los departamentos que forman parte del Eje Cafetero para el filtro
regiones_eje_cafetero = [
    ("Antioquia", "Eje Cafetero"), ("Caldas", "Eje Cafetero"), 
    ("Quindio", "Eje Cafetero"), ("Risaralda", "Eje Cafetero"), 
    ("Tolima", "Eje Cafetero"), ("Valle del Cauca", "Eje Cafetero")
]
df_regiones = spark.createDataFrame(regiones_eje_cafetero).toDF("departamento_join", "macrorregion_turistica")

# Aplicamos Broadcasting para optimizar el JOIN (Broadcasting Join)
df_regiones_broadcast = broadcast(df_regiones)
print("DataFrame de Regiones (Broadcast) listo para el Join.")

DataFrame de Regiones (Broadcast) listo para el Join.


# Persistencia en Delta-Lake

In [12]:
df_silver = (kafka_stream \
    # 2. Explosión de Metadatos
    .withColumn("value_content", from_json(col("value").cast("string"), CONTRACT_SCHEMA)) \
    .select(
        col("value_content.*"),
        col("timestamp").alias("kafka_ingestion_time"), # Metadato de Kafka
        col("offset").alias("kafka_offset")             # Metadato de Kafka
    ) \
    
    # 3. Tarea: Limpieza (Eliminación de Redundantes)
    .drop("nit_entidad", "localizacion", "sector", "es_pyme", "valor_facturado", "urlproceso") \
    
    # 3. Tarea: Cruce con Regiones (Broadcasting Join)
    .join(
        df_regiones_broadcast,
        on=df_regiones_broadcast.departamento_join == col("departamento"),
        how="inner" # INNER JOIN garantiza que solo pasen los del Eje Cafetero
    ) \
    .drop("departamento_join") \
    
    # Limpieza final de valores (solo si es necesario para asegurar tipos, aunque ya se hizo en el productor)
    .withColumn("departamento", col("departamento").cast(StringType())) \
    .withColumn("processing_time", current_timestamp())
)

# Persistencia Delta-Lake

In [13]:
# Rutas para el almacenamiento Delta Lake
DELTA_LAKE_PATH = "/opt/spark/data/delta/silver_contracts"
CHECKPOINT_PATH = "/opt/spark/data/checkpoints/silver_contracts"

# 4. Tarea: Guardar los datos limpios en formato Delta Lake
query = (df_silver.writeStream \
    .format("delta") \
    .outputMode("append") # Añadir nuevos registros
    .option("checkpointLocation", CHECKPOINT_PATH) # Obligatorio para Spark Streaming
    .option("path", DELTA_LAKE_PATH) 
    .trigger(processingTime='10 seconds') # Procesa nuevos datos cada 10 segundos
    .start()
)

print(f"Escritura del Stream a Delta Lake iniciada. Estado del Query ID: {query.id}")
print("El Job está corriendo. Presiona el botón de 'Stop' o interrupción del kernel en Jupyter para detenerlo.")

Escritura del Stream a Delta Lake iniciada. Estado del Query ID: a7f9a0a5-092b-4559-8493-d34331a9647b
El Job está corriendo. Presiona el botón de 'Stop' o interrupción del kernel en Jupyter para detenerlo.


In [14]:
## CÓDIGO DE VERIFICACIÓN DE DELTA LAKE

DELTA_LAKE_PATH = "/opt/spark/data/delta/silver_contracts"

# 1. Leer el contenido guardado en Delta Lake
df_silver_check = spark.read.format("delta").load(DELTA_LAKE_PATH)

print("--- Esquema Final de la Tabla Silver ---")
# 2. Verificar el Esquema (Solo debe tener las 8 columnas clave + metadatos)
df_silver_check.printSchema()

print("\n--- Primeros 5 Contratos del Eje Cafetero ---")
# 3. Mostrar los primeros registros para inspección
# Debes ver las columnas clave (id_contrato, valor_contrato, macrorregion_turistica)
df_silver_check.show(5, truncate=False)

print(f"\nTotal de Contratos Guardados: {df_silver_check.count()}")

# 4. Confirmación del Filtro Regional
# Verificamos que la columna macrorregion_turistica SIEMPRE sea 'Eje Cafetero'
df_filtered_region = df_silver_check.groupBy("macrorregion_turistica").count()
df_filtered_region.show()

--- Esquema Final de la Tabla Silver ---
root
 |-- id_contrato: string (nullable = true)
 |-- objeto_contrato: string (nullable = true)
 |-- entidad: string (nullable = true)
 |-- codigo_unspsc: string (nullable = true)
 |-- duracion_dias: long (nullable = true)
 |-- valor_contrato: double (nullable = true)
 |-- fecha_firma: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- kafka_ingestion_time: timestamp (nullable = true)
 |-- kafka_offset: long (nullable = true)
 |-- macrorregion_turistica: string (nullable = true)
 |-- processing_time: timestamp (nullable = true)


--- Primeros 5 Contratos del Eje Cafetero ---
+------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------+-----